# Project #1 Dealing with Multiple Pages using Selenium (Implicit vs Explicit)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

options = Options()
options.headless = False  #headless mode activated
#options.add_argument("--headless=new")  #small screen, less data. Bigger screen, more data

driver = webdriver.Chrome(options=options)
web = 'https://www.audible.com/adblbestsellers?ref_pageloadid=not_applicable&ref=a_search_t1_navTop_pl0cg1c0r0&pf_rd_p=6fb0ac98-e9fb-4acd-8278-aa237978ed3e&pf_rd_r=NCJ5CTZRYP42WJZXR010&pageLoadId=KQFN4C3hYOhbqqEG&ref_plink=not_applicable&creativeId=1642b4d1-12f3-4375-98fa-4938afc1cedc'
driver.get(web)
# driver.maximize_window()

In [2]:
#pagination
pagination = driver.find_element(By.XPATH, '//ul[contains(@class, "pagingElements")]')
pages = pagination.find_elements(By.TAG_NAME, 'li')
last_page = int(pages[-2].text)

current_page = 1

book_title = []
book_author = []
book_length = []

while current_page <= last_page:
    # time.sleep(2) # stop for 2 seconds - implicit waits

    #Explicit Waits
    container = WebDriverWait(driver,5).until(EC.presence_of_element_located(By.CLASS_NAME,'adbl-impression-container'))
    # container = driver.find_element(By.CLASS_NAME, 'adbl-impression-container')


    products = WebDriverWait(container,5).until(EC.presence_of_all_elements_located(By.XPATH,'.//li[contains(@class, "productListItem")]'))
    # products = container.find_elements(By.XPATH, './/li[contains(@class, "productListItem")]')

    for product in products:
        book_title.append(product.find_element(By.XPATH, './/h3[contains(@class, "bc-heading")]').text)
        book_author.append(product.find_element(By.XPATH, './/li[contains(@class, "authorLabel")]').text)
        book_length.append(product.find_element(By.XPATH, './/li[contains(@class, "runtimeLabel")]').text)

    current_page = current_page + 1

    try:
        next_page = driver.find_element(By.XPATH, '//span[contains(@class, "nextButton")]')
        next_page.click()
    
    except:
        pass

In [3]:
df_books= pd.DataFrame({'title': book_title, 'author': book_author, 'length':book_length})
df_books.to_csv('6_5-1_Implicit_Explicit.csv', index=False)